Code by Marvin Kinz, m.kinz@stud.uni-heidelberg.de
# Random Object Generation

The main parameters that result in differently shaped objects and that you will probably want to adjust are first "dr", which should be in the range (0,0.5) and controls the extent of deformation, with a lower value leading to more roundish shapes and a higher value leading to more deformation. Second is "it" which controls the detail of the created object and should be an integer >1. Be aware that computing time and file size increase exponentially with this parameter. Third is the smoothness parameter "sm", which governs the how many small "bumps" your object will have in the end. It is in the range [0,inf), where 0 leads to maximum displacement in each iteration and inf would mean no displacement. You could also exchange the shape that you start the generation from by handing your own over in "input_shape" if you want, but do not choose too detailed shapes, as then the generated objects will be pretty similar to them. 

The parameter for controlling how many objects you get is "nr". Another parameter which you might want to adjust is "folder", which controls the output directory where you will find your created files. The folder gets created automatically, so you do not have to create it by hand beforehand. Inside you will find two folders named "Mesh" and "PC" which contain the meshes and the point clouds of the created files respectively, latter only after uncommenting the point clouds again.

To set the parameters you can either write them in the correct order in the function call without skipping any of them or you can write i.e. "nr=10", which will then set the parameter "nr" to 10. You can also mix those two kinds, i.e. first following the correct order and then switching to naming the parameters once you have to skip some.

Below the example calls there is also an example how one can use PyVista to load and show the generated data again, if one is interested in that.

## Main function to create objects
As this is a notebook you have to run the cells before the calls.

In [ ]:
import numpy as np
import pyvista as pv
import time
import os
def rand3d(dr,it,sc=100,sm=0.5,nr=1,folder="1 Random Objects",input_shape=pv.Box()):
    """
    Creates a random 3d objects based on an input shape.
    For saving in other fileformats or applying more smoothing see the lines that are commented out in the code.

    params
    ------
    dr : double (0,0.5)
        initial displacement range: bigger value more displacement/sharper shape. Should be below 0.5, can lead to intersections otherwise
    it : int [1,inf)
        Number of iterations to randomly deform object using subdivision. Bigger number means more small scale deformations and thus more details, bigger files.
    sc : double (0,inf)
        Scaling parameter, rescales object. Without scaling the size of the object would be of order 1, which might be too small for use in some applications.
    sm : double [0,inf)
        Smoothness parameter. If >0 leads to less relative deformation per iteriation and thus smoothing the surface.
    nr : int [1,inf)
        Number of objects to generate
    folder : string
        Folder to save objects in
    input_shape : PolyData Mesh
        Shape to start deformation process from.

    returns
    -------
    Stores generated objects.
    """

    ident="main"#f'dr{dr}_it{it}_sc{sc}_sm{sm}_main'.replace(".", "_") #object identifier, if wanted stores settings in filename
    for j in range(nr): #create as many objects as wanted
        tic = time.perf_counter() #take time
        obj = input_shape.copy().triangulate() #triangulate for subdivision
        obj.points*=sc #apply scaling to obj
        for i in range(it): #iteratively deform object
            #Calculate displacement to fit subdivision.
            #Take distance from each vertex to each vertex
            norm=np.linalg.norm(obj.points-obj.points[:,np.newaxis],axis=2)
            #Set distance of vertex to itself to inf
            norm[norm==0]=np.inf
            #Get closest vertex for each vertex and multiply with displacement range scaled to iteration
            disp=np.min(norm,axis=0)*dr/(i+1)**(sm)
            #for each vertex
            for k, dis in enumerate(disp):
                #generate random displacement vector
                disp_vec=np.random.uniform(-dis,dis,3)
                #and if the vector is outside of the allowed radius
                while np.linalg.norm(disp_vec)>dis:
                    #discard it and generate a new one
                    disp_vec=np.random.uniform(-dis,dis,3)
                #apply displacement to vertex, make it smaller depending on iteration and sm
                obj.points[k]+=disp_vec
            #subdivide to make it smooth and allow for smaller displacement in next iteration
            obj.subdivide(1, subfilter='loop', inplace=True) 
        #obj.subdivide(1, subfilter='loop', inplace=True) #final smoothing, remove comment if wanted. Can also change 1 to some higher value for more smoothing
        
        #check whether directories exists and if not create them
        if not os.path.exists(folder):
            os.makedirs(folder)

        #save mesh
        if not os.path.exists(f'{folder}/Mesh'):
            os.makedirs(f'{folder}/Mesh')
        #obj.save(f'{folder}{ident}{j}_mesh.vtk') #vtk fileformat
        pv.save_meshio(f'{folder}/Mesh/{ident}{j}_mesh.obj', obj, 'obj') #obj fileformat

        #save point cloud 
        #if not os.path.exists(f'{folder}/PC'):
        #    os.makedirs(f'{folder}/PC')
        #centers = obj.cell_centers() #could also use vertices instead for point cloud, i.e. obj.points
        #centers.save(f'{folder}PC/{ident}{j}_pc.vtk') #vtk fileformat
        #centers.save(f'{folder}PC/{ident}{j}_pc.ply') #ply fileformat
        #p = pv.Plotter()
        #p.add_mesh(centers)
        #p.export_obj(f'{folder}/PC/{ident}{j}_pc') #obj fileformat

        toc = time.perf_counter()
        print(f"{ident}{j} Finished in {toc - tic:0.4f} seconds")

In [ ]:
#example for just creating many objects with standard settings
rand3d(dr=0.45,it=5,nr=10)

In [ ]:
#example batch creation with some changing settings
DR=[0.05,0.15,0.25,0.35,0.45]
IT=[1,2,3,4,5,6]
SM=[0,0.5,1,1.5,2,2.5]

for it in IT:
    print(it)
    rand3d(0.45,it,folder="Batch it")
for dr in DR:
    print(dr)
    rand3d(dr,5,folder="Batch dr")
for sm in SM:
    print(sm)
    rand3d(0.45,5,sm=sm,folder="Batch sm")


## Example how to load and show saved test object

In [ ]:
#change this to fit your actual data
ident='main0' 
folder="1 Random Objects"
#------------------------------------
test=pv.PolyData(f'{folder}/Mesh/{ident}_mesh.obj')
p = pv.Plotter()
p.add_mesh(test)
p.show(use_ipyvtk=True)